# DLS Calculator

In [7]:
import pandas as pd
from math import floor

In [8]:
# Balls/Overs Conversion

Balls_per_over=6
def overs_to_balls(overs):
    whole_overs = int(overs)
    balls_part = ((overs - whole_overs) * 10)

    if balls_part < 0 or balls_part > 5:
        raise ValueError(f"Invalid overs notation: {overs}. "
                         f"The decimal part must be between .0 and .5")
    total_balls = whole_overs * Balls_per_over + balls_part
    return total_balls

def balls_to_overs(balls):
    if balls < 0:
        raise ValueError("Balls cannot be negative")
    whole_overs = balls // BBalls_per_over
    balls_part = balls % Balls_per_over
    return float(f"{whole_overs}.{balls_part}")

def remaining_overs(max_overs,overs_bowled):
    total_balls=max_overs*Balls_per_over
    total_balls_bowled=overs_to_balls(overs_bowled)
    remaining_balls=total_balls-total_balls_bowled
    if remaining_balls < 0:
        remaining_balls = 0
    return balls_to_overs(remaining_balls)

def new_total_overs_after_delay(original_overs,delay_minutes,minutes_per_over):
    if original_overs < 0:
        raise ValueError("original_overs cannot be negative")
    if delay_minutes < 0:
        raise ValueError("delay_minutes cannot be negative")
    overs_lost = int(delay_minutes // minutes_per_over)
    new_total = original_overs - overs_lost
    return max(new_total, 0)

In [9]:
odi_df=pd.read_csv('dls_odi.csv')
odi_df.set_index("Overs_Available", inplace=True)

t20_df=pd.read_csv('dls_t20.csv')
t20_df.set_index("Overs_Available", inplace=True)

def get_resource(overs_left, wickets_lost, Format):
    Format=Format.lower()
    if Format == "odi":
        table = odi_df
    elif Format== "t20":
        table = t20_df
    else:
        raise ValueError("Invalid format_type. Use 'odi' or 't20'.")

    try:
        return table.loc[overs_left, wickets_lost]
    except KeyError:
        raise ValueError(f"No resource value for overs_left={overs_left}, wickets_lost={wickets_lost}")


In [10]:
def team2_overs_from_delay(Format, delay_minutes, minutes_per_over):
    Format=Format.lower()
    if Format == "odi":
        original_overs = 50
    elif Format  == "t20":
        original_overs = 20
    else:
        raise ValueError("match_format must be 'odi' or 't20'")

    if delay_minutes < 0:
        raise ValueError("delay_minutes cannot be negative")
    if minutes_per_over <= 0:
        raise ValueError("minutes_per_over must be positive")

    overs_lost = int(delay_minutes // minutes_per_over)
    new_total = original_overs - overs_lost

    # Don't go below 0
    return max(new_total, 0)


In [14]:
def revised_target_from_delay(team1_score,Format ,delay_minutes,minutes_per_over):
    Format=Format.lower()
    # 1) Team 2 overs from delay
    team2_overs = team2_overs_from_delay(Format, delay_minutes, minutes_per_over)

    if Format== "odi":
        table = odi_df
        scheduled_overs = 50
    elif Format  == "t20":
        table = t20_df
        scheduled_overs = 20
    else:
        raise ValueError("match_format must be 'odi' or 't20'")
    col0 = "0"
    # 2) Resources
    R1 = float(table.loc[scheduled_overs, col0])   # Team 1 full resources (0 wickets, full overs) ~100
    R2 = float(table.loc[team2_overs, col0])       # Team 2 starts at 0 wickets

    par = team1_score * (R2 / R1)
    target = floor(par) + 1
    rrr=round(target/team2_overs,2)
    return target, team2_overs,rrr


In [15]:
def main():
    Format = input("Match format (odi/t20): ").strip().lower()
    team1_score= int(input("Team 1 score: ").strip())
    delay_minutes = int(input("Rain delay before Team 2 (in minutes): ").strip())
    minutes_per_over=4.0
    target, team2_overs,rrr = revised_target_from_delay(team1_score,Format,delay_minutes,minutes_per_over)

    print("\n--- DLS-style Result (Simplified) ---")
    print(f"Team 2 overs available : {team2_overs}")
    print(f"Revised target for Team 2: {target} runs")
    print(f"Required Run Rate : {rrr}")

if __name__ == "__main__":
    main()


Match format (odi/t20):  t20
Team 1 score:  190
Rain delay before Team 2 (in minutes):  15



--- DLS-style Result (Simplified) ---
Team 2 overs available : 17
Revised target for Team 2: 167 runs
Required Run Rate : 9.82


In [6]:
import pandas as pd
from math import floor


# Balls/Overs Conversion

Balls_per_over=6
def overs_to_balls(overs):
    whole_overs = int(overs)
    balls_part = ((overs - whole_overs) * 10)

    if balls_part < 0 or balls_part > 5:
        raise ValueError(f"Invalid overs notation: {overs}. "
                         f"The decimal part must be between .0 and .5")
    total_balls = whole_overs * Balls_per_over + balls_part
    return total_balls

def balls_to_overs(balls):
    if balls < 0:
        raise ValueError("Balls cannot be negative")
    whole_overs = balls // BBalls_per_over
    balls_part = balls % Balls_per_over
    return float(f"{whole_overs}.{balls_part}")

def remaining_overs(max_overs,overs_bowled):
    total_balls=max_overs*Balls_per_over
    total_balls_bowled=overs_to_balls(overs_bowled)
    remaining_balls=total_balls-total_balls_bowled
    if remaining_balls < 0:
        remaining_balls = 0
    return balls_to_overs(remaining_balls)

def new_total_overs_after_delay(original_overs,delay_minutes,minutes_per_over):
    if original_overs < 0:
        raise ValueError("original_overs cannot be negative")
    if delay_minutes < 0:
        raise ValueError("delay_minutes cannot be negative")
    overs_lost = int(delay_minutes // minutes_per_over)
    new_total = original_overs - overs_lost
    return max(new_total, 0)


odi_df=pd.read_csv('dls_odi.csv')
odi_df.set_index("Overs_Available", inplace=True)

t20_df=pd.read_csv('dls_t20.csv')
t20_df.set_index("Overs_Available", inplace=True)

def get_resource(overs_left, wickets_lost, Format):
    Format=Format.lower()
    if Format == "odi":
        table = odi_df
    elif Format== "t20":
        table = t20_df
    else:
        raise ValueError("Invalid format_type. Use 'odi' or 't20'.")

    try:
        return table.loc[overs_left, wickets_lost]
    except KeyError:
        raise ValueError(f"No resource value for overs_left={overs_left}, wickets_lost={wickets_lost}")



def team2_overs_from_delay(Format, delay_minutes, minutes_per_over):
    Format=Format.lower()
    if Format == "odi":
        original_overs = 50
    elif Format  == "t20":
        original_overs = 20
    else:
        raise ValueError("match_format must be 'odi' or 't20'")

    if delay_minutes < 0:
        raise ValueError("delay_minutes cannot be negative")
    if minutes_per_over <= 0:
        raise ValueError("minutes_per_over must be positive")

    overs_lost = int(delay_minutes // minutes_per_over)
    new_total = original_overs - overs_lost

    # Don't go below 0
    return max(new_total, 0)



def revised_target_from_delay(team1_score,Format ,delay_minutes,minutes_per_over):
    Format=Format.lower()
    # 1) Team 2 overs from delay
    team2_overs = team2_overs_from_delay(Format, delay_minutes, minutes_per_over)

    if Format== "odi":
        table = odi_df
        scheduled_overs = 50
    elif Format  == "t20":
        table = t20_df
        scheduled_overs = 20
    else:
        raise ValueError("match_format must be 'odi' or 't20'")
    col0 = "0"
    # 2) Resources
    R1 = float(table.loc[scheduled_overs, col0])   # Team 1 full resources (0 wickets, full overs) ~100
    R2 = float(table.loc[team2_overs, col0])       # Team 2 starts at 0 wickets

    par = team1_score * (R2 / R1)
    target = floor(par) + 1
    rrr=round(target/team2_overs,2)
    return target, team2_overs,rrr



def main():
    Format = input("Match format (odi/t20): ").strip().lower()
    team1_score= int(input("Team 1 score: ").strip())
    delay_minutes = int(input("Rain delay before Team 2 (in minutes): ").strip())
    minutes_per_over=4.0
    target, team2_overs,rrr = revised_target_from_delay(team1_score,Format,delay_minutes,minutes_per_over)

    print("\n--- DLS-style Result (Simplified) ---")
    print(f"Team 2 overs available : {team2_overs}")
    print(f"Revised target for Team 2: {target} runs")
    print(f"Required Run Rate : {rrr}")

if __name__ == "__main__":
    main()


Match format (odi/t20):  odi
Team 1 score:  300
Rain delay before Team 2 (in minutes):  30



--- DLS-style Result (Simplified) ---
Team 2 overs available : 43
Revised target for Team 2: 279 runs
Required Run Rate : 6.49
